In [ ]:
import numpy as np

with open('input.txt', 'r') as f:
  lines = f.read().splitlines()

In [ ]:
maze = np.array([list(l) for l in lines])

# Find starting point to initialize frontier
frontier = [(i, j) for i in range(maze.shape[0])
                   for j in range(maze.shape[1])
                   if maze[i, j] == 'S']

i_start, j_start = frontier[0]

# Prepare distance and visited matrix
visited = np.zeros(maze.shape)
dist = np.ones(maze.shape) * np.inf
dist[i_start, j_start] = 0

# Replace S with actual symbol it should be
def get_start_replacement():
  connected_north = i_start > 0 and maze[i_start - 1, j_start] in ['|', '7', 'F']
  connected_south = i_start < maze.shape[0] - 1 and maze[i_start + 1, j_start] in ['|', 'L', 'J']
  connected_west = j_start > 0 and maze[i_start, j_start - 1] in ['-', 'L', 'F']
  connected_east = j_start < maze.shape[1] - 1 and maze[i_start, j_start + 1] in ['-', 'J', '7']

  if connected_north and connected_south:
    return '|'

  if connected_east and connected_west:
    return '-'

  if connected_north and connected_east:
    return 'L'

  if connected_north and connected_west:
    return 'J'

  if connected_south and connected_west:
    return '7'

  if connected_south and connected_east:
    return 'F'

maze[i_start, j_start] = get_start_replacement()

def neighbors(i, j):
  result = []

  north = (i - 1, j)
  south = (i + 1, j)
  west = (i, j - 1)
  east = (i, j + 1)

  if maze[i, j] == '|':
    result.extend([north, south])
  elif maze[i, j] == '-':
    result.extend([west, east])
  elif maze[i, j] == 'L':
    result.extend([north, east])
  elif maze[i, j] == 'J':
    result.extend([north, west])
  elif maze[i, j] == '7':
    result.extend([south, west])
  elif maze[i, j] == 'F':
    result.extend([south, east])

  filtered = [(i_prime, j_prime)
              for (i_prime, j_prime) in result
              if 0 <= i_prime < maze.shape[0] and 0 <= j_prime < maze.shape[1]]

  return filtered

farthest_distance = 0

while frontier:
  # Pop entry with smallest tentative distance
  current = sorted(frontier, key=lambda x: dist[x])[0]
  frontier.remove(current)

  # Mark as visited
  visited[*current] = 1

  # Get neighbors
  nn = neighbors(*current)
  # print(f'current: {current} - neighbors: {nn}')

  for (i_n, j_n) in nn:
    dist[i_n, j_n] = min(dist[i_n, j_n], dist[*current] + 1)

    # Keep track of the farthest distance we've ever travelled from S
    farthest_distance = max(farthest_distance, dist[i_n, j_n])

    # Add to frontier if not yet visited
    if (visited[i_n, j_n] == 0):
      frontier.append((i_n, j_n))

print(farthest_distance)

In [ ]:
# For part 2, determine points enclosed within the loop via the standard even/odd algorithm.
# To avoid having to deal with pipes that run in parallel to the "scan lines" used by the
# algorithm, trace in diagonal direction from the top/left edges of the maze area.
# This way, we'll have to consider only '|', '-', 'J', 'F' (for other corners, we'll just
# "scrape by", never intersecting).

inout = np.zeros(maze.shape)

rays = [(i, 0) for i in range(maze.shape[0])] + [(0, j) for j in range(1, maze.shape[1])]

for ray in rays:
  i, j = ray
  inside = False
  while i < maze.shape[0] and j < maze.shape[1]:
    if visited[i, j] != 0:
      if maze[i, j] in ['|', '-', 'J', 'F']:
        inside = not inside
    else:
      if inside:
        inout[i, j] = 1

    i += 1
    j += 1

np.sum(inout, dtype=int)